In [1]:
!unzip Finance.zip

Archive:  Finance.zip
  inflating: FinancialPhraseBank/License.txt  
  inflating: FinancialPhraseBank/README.txt  
  inflating: FinancialPhraseBank/Sentences_50Agree.txt  
  inflating: FinancialPhraseBank/Sentences_66Agree.txt  
  inflating: FinancialPhraseBank/Sentences_75Agree.txt  
  inflating: FinancialPhraseBank/Sentences_AllAgree.txt  
  inflating: all-data.csv            


In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2023-05-05 09:16:28--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-05-05 09:16:28--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-05 09:16:29--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [3]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [15]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.stem import PorterStemmer
nltk.download('wordnet')
import string
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
df=pd.read_csv("all-data.csv", encoding = "ISO-8859-1", names =['sentiment' , 'text'])
X=df["text"]
y=df["sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def load_glove_model(File):
    print("Loading Glove Model")
    glove_model = {}
    with open(File,'r') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    print(f"{len(glove_model)} words loaded!")
    return glove_model
vectors=load_glove_model('glove.6B.100d.txt')

Loading Glove Model
400000 words loaded!


In [13]:
contractions_dict = {"ain't": "are not","'s":" is","aren't": "are not"}
# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)
# Expanding Contractions in the reviews
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopwords_set])

stopwords_set = set(stopwords.words('english'))
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

In [16]:
X=df["text"]
y=df["sentiment"]
X=X.apply(lambda x: x.lower())
X=X.apply(lambda x:expand_contractions(x))
X=X.apply(lambda x: re.sub('W*dw*','',x))
X=X.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
#X=X.apply(lambda x: remove_stopwords(x))
X = X.apply(lambda x: stem_words(x))
X = X.apply(lambda text: lemmatize_words(text))
X= X.apply(lambda x: re.sub(' +', ' ', x))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [17]:
def calculate_vec(X):
  py_nlp_en = English()
  py_tokenizer_en = py_nlp_en.tokenizer 
  value_vector=0
  feature_map=[]
  for text in X:
    words =list(py_tokenizer_en(text))
    value_vector=np.zeros(100)
    temp=0
    for word in words:
      word=str(word.text)
      if word in vectors:
        value_vector=vectors[word]+value_vector
        temp=temp+1
    feature_map.append(value_vector/temp)
  return feature_map
feature_maps_train=calculate_vec(X_train)
feature_maps_test=calculate_vec(X_test)

In [18]:
def labler(y):
  label_to_int = {"neutral": 1, "positive": 2, "negative": 0}
  output_data = list()
  for label in y:
    output_data.append(label_to_int[label])
  output_data = np.array(output_data)
  return output_data

In [19]:
y_train=labler(y_train)
y_test=labler(y_test)

In [20]:
from sklearn.metrics import classification_report

In [21]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='sag', multi_class='multinomial')
lr.fit(feature_maps_train, y_train)
y_pred = lr.predict(feature_maps_train)
print(classification_report(y_train, y_pred))

y_pred = lr.predict(feature_maps_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.31      0.41       543
           1       0.72      0.91      0.81      2606
           2       0.60      0.40      0.48      1212

    accuracy                           0.69      4361
   macro avg       0.64      0.54      0.56      4361
weighted avg       0.67      0.69      0.66      4361

              precision    recall  f1-score   support

           0       0.80      0.26      0.40        61
           1       0.68      0.91      0.78       273
           2       0.69      0.46      0.55       151

    accuracy                           0.69       485
   macro avg       0.72      0.54      0.58       485
weighted avg       0.70      0.69      0.66       485



In [22]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(feature_maps_train, y_train)
y_pred = clf.predict(feature_maps_train)
print(classification_report(y_train, y_pred))

y_pred = clf.predict(feature_maps_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.29      0.52      0.37       543
           1       0.71      0.75      0.73      2606
           2       0.36      0.19      0.25      1212

    accuracy                           0.57      4361
   macro avg       0.46      0.49      0.45      4361
weighted avg       0.56      0.57      0.55      4361

              precision    recall  f1-score   support

           0       0.28      0.48      0.35        61
           1       0.67      0.78      0.72       273
           2       0.48      0.19      0.27       151

    accuracy                           0.56       485
   macro avg       0.47      0.48      0.45       485
weighted avg       0.56      0.56      0.54       485

